# American Sign Languege (ASL)
นำ Dataset มาจากเว็บไซต์ Kaggle : 
[ASL(American Sign Language) Alphabet Dataset](https://www.kaggle.com/datasets/debashishsau/aslamerican-sign-language-aplhabet-dataset)

การนำ Dataset จากเว็บไซต์ Kaggle มาใช้ต้องอัปโหลด **kaggle.json** เข้าไปที่ `... /root/ .kaggle` 

[วิธีอัปโหลด Dataset จากเว็บไซต์ Kaggle]( https://www.youtube.com/watch?v=57N1g8k2Hwc)

In [ ]:
! pip install kaggle

In [ ]:
# สร้างโฟลเดอร์ .kaggle เเละอัปโหลด kaggle.json
! mkdir ./.kaggle
! cp kaggle.json ./.kaggle/
! chmod 600 ./.kaggle/kaggle.json

In [ ]:
# อัปโหลด API เพื่อดาวโหลด Dataset ใน kaggle
! kaggle datasets download -d debashishsau/aslamerican-sign-language-aplhabet-dataset

In [ ]:
# unzip Dataset
! unzip aslamerican-sign-language-aplhabet-dataset.zip -d data

In [ ]:
! pip install torchvision
! pip install pytorch-lightning

In [ ]:
import os
import os.path as op
import shutil
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from torchvision import datasets, models, transforms
import torchvision.transforms as T
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch import nn
import torch
import PIL
from os import listdir
from os.path import isfile, join
from torch.utils.data import Dataset
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import seaborn as sns

In [ ]:
# ดึงรูปจากใน class train
class CustomDataset(Dataset):
    def __init__(self, train_dir,transformation):
        self.img_labels = []
        self.img_dirs = []
        self.transformation = transformation
        
        for label in listdir(train_dir):
            img_dirs_in_folder = [join(train_dir, label, file_name) for file_name in listdir(join(train_dir, label)) if isfile(join(train_dir, label, file_name))]
            self.img_dirs.extend(img_dirs_in_folder)
            self.img_labels.extend([label] * len(img_dirs_in_folder))
            
        self.labels_encode = {class_name: i for i , class_name in enumerate(sorted(set(self.img_labels)))}    
    def __len__(self):
        return len(self.img_dirs)
    
    def _read_img(self, img_dir):
        return PIL.Image.open(img_dir)
    
    def __getitem__(self, index):
        img = self._read_img(self.img_dirs[index])
        if self.transformation : 
          img = self.transformation(img)
        label = self.img_labels[index]
        label = self.labels_encode[label]
        return img, label

In [ ]:
# ปรับขนาดรูปภาพให้เป็น 224*224
transformation = transforms.Compose([
  transforms.Resize((224,224)),
  transforms.ToTensor(),
  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])                                   
])

In [ ]:
train[0][0].shape

torch.Size([3, 224, 224])

In [ ]:
# Dataset ทั้งหมด 29 class
train_dataset = CustomDataset("/content/data/ASL_Alphabet_Dataset/asl_alphabet_train",transformation = transformation)
print(train_dataset.labels_encode)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 'del': 26, 'nothing': 27, 'space': 28}


# Data loader

In [ ]:
# สร้าง pretrained model เป็น resnet34
model = models.resnet34(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

In [ ]:
# Model ที่เอามา Train ต่อ
# model = torch.load(f"/content/drive/MyDrive/ASL_20classify.pt")

In [ ]:
#(fc): Linear(in_features=512, out_features=1000, bias=True) มาจาก print(model)
model.fc = nn.Linear(512, 29)

In [ ]:
 torch.cuda.is_available()

True

In [ ]:
# จำนวนข้อมูลทั้งหมด
len(train_dataset)

223074

In [ ]:
# เเบ่ง train ,test
train,test = torch.utils.data.random_split(train_dataset,[223074-20000,20000])
train_loader = DataLoader(train, batch_size=128, shuffle=True)
test_loader = DataLoader(test, batch_size=128, shuffle=False)

In [ ]:
# สร้าง loss, optimizer
cross_entropy = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=2e-3)

In [ ]:
# เช็คว่ามี GPU ที่สามารถใช้ได้มั้ย ถ้าใช้ได้นำโมเดลเข้าไปอยู่ใน GPU
gpu = torch.cuda.is_available()
print(gpu)
if gpu:
    model.cuda()

True


# Train Dataset

In [ ]:
n_train = len(train_loader.dataset)
n_val = len(test_loader.dataset)

In [ ]:
n_epochs = 1 # จำนวนรอบ
for epoch in range(n_epochs):
    # ช่วง train
    model.train()
    train_loss, val_loss = 0, 0
    for images, labels in tqdm(train_loader):
        if gpu:
            images, labels = images.cuda(), labels.cuda()
        optimizer.zero_grad()
        pred = model(images) # คำนวณหา output (pred) จากโมเดลที่มีอยู่
        loss = cross_entropy(pred, labels)
        loss.backward() # คำนวณ gradient จาก loss ที่ได้
        optimizer.step() # อัพเดทพารามิเตอร์ของโมเดล
        train_loss += loss.item() * images.size(0)

    # ช่วง validate
    model.eval() # เซ็ตเป็น evaluation mode
    torch.save(model,f"/content/drive/MyDrive/ชื่อไฟล์.pt") # save model
    for images, labels in tqdm(test_loader):
        if gpu:
            images, labels = images.cuda(), labels.cuda()
        pred = model(images)
        loss = cross_entropy(pred, labels)
        val_loss += loss.item() * images.size(0)
    print("Training loss = {}, Validation loss = {}".format(train_loss / n_train, val_loss / n_val))

# Confusion matrix

In [ ]:
# ใส่ Path โมเดล
new_model = torch.load(f"...")#โหลดโมเดล
new_model.eval()

In [ ]:
pred_list = []
label_list = []
for data, label in tqdm(test_loader):
  pred = new_model(data.cuda()).cpu().detach()
  pred = torch.argmax(torch.softmax(pred,dim=1),dim=1)
  pred_list.extend(pred.numpy().tolist())
  label_list.extend(label.numpy().tolist())

In [ ]:
label_encode = {
        'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 'del': 26, 'nothing': 27, 'space': 28
    }
label_decode = {value: key for key, value in label_encode.items()}

pred_key = [label_decode[pred] for pred in pred_list]
label_key = [label_decode[label] for label in label_list]
labels = list(label_encode.keys())

In [ ]:
cfm = confusion_matrix(label_key ,pred_key, labels = list(label_encode.keys()))

In [ ]:
# ความเเม่นยำ(Accuracy) ของทุกคลาส ,ทั้งหมด 
df_cm = pd.DataFrame(cfm/np.sum(cfm, axis = -1) *100, index = [i for i in labels],columns = [i for i in labels])
plt.figure(figsize = (12*2,7*2))
sns.heatmap(df_cm, annot=True)
plt.show()

In [ ]:
# ความเเม่นยำ(Accuracy) ของทุกคลาส ,ทั้งหมด 
rint(classification_report(label_key ,pred_key, labels = list(label_encode.keys())))

# ทำนายผลจาก Test ข้างนอก (Predict)

In [ ]:
# ใส่ Path ภาพ
images = '...' 

# ใส่ Path โมเดล
new_model = torch.load(f"...")
new_model.eval()

In [ ]:
import cv2
def predict(img_path, model):
    def Get_img(images_path, transformation):
        img = PIL.Image.open(images_path)
        if transformation : 
          img = transformation(img)
        return img

    img = Get_img(img_path, transformation = transformation)
    img = img[None, :, :, :]
    label_encode = {
        'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 'del': 26, 'nothing': 27, 'space': 28
    }
    label_decode = {value: key for key, value in label_encode.items()}
    prediction = model(img.cuda())
    print('Predicted As :',label_decode[int(prediction.argmax())])
    images = cv2.imread(img_path)
    images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB)
    plt.imshow(images)
    
    return torch.softmax(prediction,dim=1)

In [ ]:
prediction = predict(images, new_model)
print(torch.round(prediction * 100,decimals=1))